In [1]:
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

For the data to be processed, we need to do some cleaning and transformations.

- Convert date format
- Change Vol. from percentage to real numbers
- Change values from million, and K to real numbers
- Changing data type from Object to float for some of the columns

In [2]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%b %d, %Y')
# loading data for Argentina
argentina_df = pd.read_csv('data/argentina.csv',sep=',', parse_dates=['Date'], date_parser=dateparse).fillna(0)
argentina_df['Vol.'] = (argentina_df['Vol.'].replace(r'[KM]+$', '', regex=True)
                           .astype(float) * argentina_df['Vol.'].str.extract(r'[\d\.]+([KM]+)', expand=False)
                           .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))

# loading data for Brazil
brazil_df = pd.read_csv('data/brazil.csv',sep=',', parse_dates=['Date'], date_parser=dateparse).fillna(0)
brazil_df['Vol.'] = (brazil_df['Vol.'].replace(r'[KM]+$', '', regex=True)
                           .astype(float) * brazil_df['Vol.'].str.extract(r'[\d\.]+([KM]+)', expand=False)
                           .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))

# loading data for Colombia
colombia_df = pd.read_csv('data/colombia.csv',sep=',', parse_dates=['Date'], date_parser=dateparse).fillna(0)
colombia_df['Vol.'] = (colombia_df['Vol.'].replace(r'[KM]+$', '', regex=True)
                           .astype(float) * colombia_df['Vol.'].str.extract(r'[\d\.]+([KM]+)', expand=False)
                           .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))
colombia_df['Price'] = colombia_df['Price'].str.replace(',', '')
colombia_df['Price'] = colombia_df['Price'].astype(float)
colombia_df['Open'] = colombia_df['Open'].str.replace(',', '')
colombia_df['Open'] = colombia_df['Open'].astype(float)
colombia_df['High'] = colombia_df['High'].str.replace(',', '')
colombia_df['High'] = colombia_df['High'].astype(float)
colombia_df['Low'] = colombia_df['Low'].str.replace(',', '')
colombia_df['Low'] = colombia_df['Low'].astype(float)

# loading data for Egypt
egypt_df = pd.read_csv('data/egypt.csv',sep=',', parse_dates=['Date'], date_parser=dateparse).fillna(0)
egypt_df['Vol.'] = (egypt_df['Vol.'].replace(r'[KM]+$', '', regex=True)
                           .astype(float) * egypt_df['Vol.'].str.extract(r'[\d\.]+([KM]+)', expand=False)
                           .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))

/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/3666261852.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparse = lambda dates: pd.datetime.strptime(dates, '%b %d, %Y')
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/3666261852.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparse = lambda dates: pd.datetime.strptime(dates, '%b %d, %Y')
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/3666261852.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparse = lambda dates: pd.datetime.strptime(dates, '%b %d, %Y')
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/3666261852.py:1: FutureWarning: The pandas.date

In [3]:
# function to normalize values to feed to RNN
def normalize_data(df):
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    df['Open'] = min_max_scaler.fit_transform(df.Open.values.reshape(-1,1))
    df['High'] = min_max_scaler.fit_transform(df.High.values.reshape(-1,1))
    df['Low'] = min_max_scaler.fit_transform(df.Low.values.reshape(-1,1))
    df['Price'] = min_max_scaler.fit_transform(df.Price.values.reshape(-1,1))
    return df

In [13]:
# function for splitting data into train, test, and validation
def split_data(df):
    start_date = '2021-01-01' 
    end_date = '2021-03-31'
    val_start_date = '2020-12-01' 
    val_end_date = '2020-12-30'
    test = df.loc[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    remaining = df[~df.index.isin(test.index)]
    val_data = remaining.loc[(remaining['Date'] >= val_start_date) & (remaining['Date'] <= val_end_date)]
    train = remaining[~remaining.index.isin(val_data.index)]
    # Making Date as the index for time series problem. 
    x_train = train[['Date','Price','Open','High','Low']]
    x_train['Date'] = pd.to_datetime(train['Date'])
    x_train.set_index('Date', inplace=True)
    val_train = val_data[['Date','Price','Open','High','Low']]
    val_train['Date'] = pd.to_datetime(val_data['Date'])
    val_train.set_index('Date', inplace=True)
    y_test = test[['Date','Price','Open','High','Low']]
    y_test['Date'] = pd.to_datetime(y_test['Date'])
    y_test.set_index('Date', inplace=True)
    # For training, we're only interested in the Open, High, and Low columns. 
    X_train = np.array(x_train[['Open','High','Low']])
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    Y_train = x_train['Price']
    X_val = np.array(val_train[['Open','High','Low']])
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
    Y_val = val_train['Price']
    X_test = np.array(y_test[['Open','High','Low']])
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    Y_test = y_test['Price']
    return X_train,Y_train,X_val,Y_val,X_test,Y_test

In [15]:
# Define the model
model = keras.Sequential()
model.add(layers.GRU(64, input_shape=(3, 1), return_sequences=True))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(32, return_sequences=False))
model.add(layers.Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [20]:
# train and evaluate function
def train_eval(model,X_train, Y_train, X_val, Y_val, X_test, Y_test, period,stock_name):
    history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_val, Y_val))
    # Evaluate the model on the test set
    mae = model.evaluate(X_test[:period], Y_test[:period])
    print('MAE for '+stock_name+' for',period,' days', mae)

In [22]:
# For Argentina
normalized_data = normalize_data(argentina_df)
X_train,Y_train,X_val,Y_val,X_test,Y_test = split_data(normalized_data)
# Train the model
train_eval(model,X_train,Y_train,X_val,Y_val,X_test,Y_test,7,"Argentina")

Epoch 1/50
14/14 [==============================] - 0s 4ms/step - loss: 0.0216 - val_loss: 0.0346
Epoch 2/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0240 - val_loss: 0.0333
Epoch 3/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0214 - val_loss: 0.0233
Epoch 4/50
 1/14 [=>............................] - ETA: 0s - loss: 0.0165

/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['Date'] = pd.to_datetime(train['Date'])
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_train['Date'] = pd.to_datetime(val_data['Date'])
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

14/14 [==============================] - 0s 3ms/step - loss: 0.0229 - val_loss: 0.0245
Epoch 5/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0240 - val_loss: 0.0272
Epoch 6/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0213 - val_loss: 0.0274
Epoch 7/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0236 - val_loss: 0.0245
Epoch 8/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0216 - val_loss: 0.0276
Epoch 9/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0215 - val_loss: 0.0323
Epoch 10/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0210 - val_loss: 0.0256
Epoch 11/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0216 - val_loss: 0.0393
Epoch 12/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0240 - val_loss: 0.0266
Epoch 13/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0222 - val_loss: 0.0284
Epoch 14/50
14/14 [========

In [23]:
# For Brazil
normalized_data = normalize_data(brazil_df)
X_train,Y_train,X_val,Y_val,X_test,Y_test = split_data(normalized_data)
# Train the model
train_eval(model,X_train,Y_train,X_val,Y_val,X_test,Y_test,7,"Brazil")

Epoch 1/50
15/15 [==============================] - 0s 4ms/step - loss: 0.0293 - val_loss: 0.0135
Epoch 2/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0294 - val_loss: 0.0132
Epoch 3/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0308 - val_loss: 0.0143
Epoch 4/50
 1/15 [=>............................] - ETA: 0s - loss: 0.0289

/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['Date'] = pd.to_datetime(train['Date'])
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_train['Date'] = pd.to_datetime(val_data['Date'])
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

15/15 [==============================] - 0s 3ms/step - loss: 0.0339 - val_loss: 0.0121
Epoch 5/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0300 - val_loss: 0.0124
Epoch 6/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0293 - val_loss: 0.0159
Epoch 7/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0302 - val_loss: 0.0145
Epoch 8/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0282 - val_loss: 0.0125
Epoch 9/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0262 - val_loss: 0.0199
Epoch 10/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0270 - val_loss: 0.0156
Epoch 11/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0303 - val_loss: 0.0153
Epoch 12/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0291 - val_loss: 0.0136
Epoch 13/50
15/15 [==============================] - 0s 3ms/step - loss: 0.0278 - val_loss: 0.0129
Epoch 14/50
15/15 [========

In [24]:
# For Colombia
normalized_data = normalize_data(colombia_df)
X_train,Y_train,X_val,Y_val,X_test,Y_test = split_data(normalized_data)
# Train the model
train_eval(model,X_train,Y_train,X_val,Y_val,X_test,Y_test,7,"Colombia")

Epoch 1/50
14/14 [==============================] - 0s 4ms/step - loss: 0.0250 - val_loss: 0.0342
Epoch 2/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0206 - val_loss: 0.0143
Epoch 3/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0176 - val_loss: 0.0168
Epoch 4/50
 1/14 [=>............................] - ETA: 0s - loss: 0.0143

/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['Date'] = pd.to_datetime(train['Date'])
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_train['Date'] = pd.to_datetime(val_data['Date'])
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

14/14 [==============================] - 0s 3ms/step - loss: 0.0180 - val_loss: 0.0166
Epoch 5/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0168 - val_loss: 0.0167
Epoch 6/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0178 - val_loss: 0.0170
Epoch 7/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0183 - val_loss: 0.0176
Epoch 8/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0174 - val_loss: 0.0146
Epoch 9/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0154 - val_loss: 0.0147
Epoch 10/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0171 - val_loss: 0.0156
Epoch 11/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0168 - val_loss: 0.0180
Epoch 12/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0184 - val_loss: 0.0256
Epoch 13/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0195 - val_loss: 0.0146
Epoch 14/50
14/14 [========

In [25]:
# For Egypt
normalized_data = normalize_data(egypt_df)
X_train,Y_train,X_val,Y_val,X_test,Y_test = split_data(normalized_data)
# Train the model
train_eval(model,X_train,Y_train,X_val,Y_val,X_test,Y_test,7,"Egypt")

Epoch 1/50
14/14 [==============================] - 0s 4ms/step - loss: 0.0202 - val_loss: 0.0061
Epoch 2/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0163 - val_loss: 0.0087
Epoch 3/50
 1/14 [=>............................] - ETA: 0s - loss: 0.0114

/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['Date'] = pd.to_datetime(train['Date'])
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_train['Date'] = pd.to_datetime(val_data['Date'])
/var/folders/tc/qfc76dxj7456z8tqv6mk4xfh0000gn/T/ipykernel_33093/1071198107.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

14/14 [==============================] - 0s 3ms/step - loss: 0.0147 - val_loss: 0.0117
Epoch 4/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0137 - val_loss: 0.0062
Epoch 5/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0148 - val_loss: 0.0081
Epoch 6/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0124 - val_loss: 0.0197
Epoch 7/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0142 - val_loss: 0.0202
Epoch 8/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0154 - val_loss: 0.0064
Epoch 9/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0194 - val_loss: 0.0106
Epoch 10/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0146 - val_loss: 0.0060
Epoch 11/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0127 - val_loss: 0.0076
Epoch 12/50
14/14 [==============================] - 0s 3ms/step - loss: 0.0165 - val_loss: 0.0063
Epoch 13/50
14/14 [=========

# Classifying BUY HOLD, SELL signals

In [26]:
# Importing some essential libraries
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score

The first thing we need to do is to add an additional column 'Signal' that will contains three classes:
- 0 = HOLD
- 1 = SELL
- 2 = BUY

In [27]:
signal_func = lambda x: 0 if float(x.strip('%')) <= 0.10 and float(x.strip('%')) > 0.0 else 1 if float(x.strip('%')) > 0.10 else 2

# Apply the lambda function to create the 'Signal' column
argentina_df['Signal'] = argentina_df['Change %'].apply(signal_func)
brazil_df['Signal'] = brazil_df['Change %'].apply(signal_func)
colombia_df['Signal'] = colombia_df['Change %'].apply(signal_func)
egypt_df['Signal'] = egypt_df['Change %'].apply(signal_func)

In [28]:
egypt_df

,Date,Price,Open,High,Low,Vol.,Change %,Signal
0,2021-03-31,0.528926,0.534694,0.543568,0.539095,1570000.0,0.54%,1
1,2021-03-30,0.520661,0.522449,0.518672,0.522634,3430000.0,0.81%,1
2,2021-03-29,0.508264,0.514286,0.506224,0.526749,1250000.0,-0.27%,2
3,2021-03-28,0.512397,0.542857,0.518672,0.526749,2340000.0,0.00%,2
4,2021-03-25,0.512397,0.485714,0.526971,0.489712,2500000.0,1.37%,1
...,...,...,...,...,...,...,...,...
300,2020-01-09,0.888430,0.865306,0.908714,0.884774,2090000.0,4.76%,1
301,2020-01-08,0.801653,0.718367,0.863071,0.720165,3520000.0,3.04%,1
302,2020-01-06,0.747934,0.706122,0.800830,0.707819,2520000.0,-2.95%,2
303,2020-01-05,0.801653,0.926531,0.908714,0.794239,3760000.0,-8.32%,2


In [29]:
# create a new train and split function for classification
def split_train_test(df):
    start_date = '2021-01-01' 
    end_date = '2021-03-31'
    test = df.loc[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    train = df[~df.index.isin(test.index)]
    X_train = train[['Price','Open','High','Low']]
    y_train = train['Signal']
    X_test = test[['Price','Open','High','Low']]
    y_test = test['Signal']
    return X_train,y_train,X_test,y_test

In [34]:
# function for random forest
def random_forest(X_train,y_train,X_test,y_test,stock_name):
    # Create a Random Forest classifier with 100 trees
    rf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the classifier on the training data
    rf.fit(X_train, y_train)

    # Predict the class labels for the test data
    y_pred = rf.predict(X_test)

    # Compute the accuracy score of the classifier
    accuracy = accuracy_score(y_test, y_pred)
    print("Random Forest:: For "+stock_name+" stock")
    print(f"Accuracy: {accuracy:.2f}")

In [39]:
# function for XGBoost
def xg_boost(X_train,y_train,X_test,y_test,stock_name):
    params = {
    'objective': 'multi:softmax',
    'num_class': 3,
    'max_depth': 3,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'mlogloss',
    'seed': 42
    }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    num_rounds = 100
    model = xgb.train(params, dtrain, num_rounds)
    y_pred = model.predict(dtest)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("XGBoost:: For "+stock_name+" stock")
    print("Accuracy: {:.2f}%".format(accuracy*100))
    print('f1 score is ', f1)
    print('recall is ', recall)
    print('precision score is ', precision)

In [40]:
normalized_data = normalize_data(argentina_df)
X_train,y_train,X_test,y_test = split_train_test(normalized_data)
random_forest(X_train,y_train,X_test,y_test,"Argentina")
xg_boost(X_train,y_train,X_test,y_test,"Argentina")

Random Forest:: For Argentina stock
Accuracy: 0.65
XGBoost:: For Argentina stock
Accuracy: 63.33%
f1 score is  0.6341490545050057
recall is  0.6333333333333333
precision score is  0.6406006674082314


In [42]:
normalized_data = normalize_data(brazil_df)
X_train,y_train,X_test,y_test = split_train_test(normalized_data)
random_forest(X_train,y_train,X_test,y_test,"Brazil")
xg_boost(X_train,y_train,X_test,y_test,"Brazil")

Random Forest:: For Brazil stock
Accuracy: 0.58
XGBoost:: For Brazil stock
Accuracy: 53.33%
f1 score is  0.5191919191919193
recall is  0.5333333333333333
precision score is  0.6142857142857143


In [43]:
normalized_data = normalize_data(colombia_df)
X_train,y_train,X_test,y_test = split_train_test(normalized_data)
random_forest(X_train,y_train,X_test,y_test,"Colombia")
xg_boost(X_train,y_train,X_test,y_test,"Colombia")

Random Forest:: For Colombia stock
Accuracy: 0.57
XGBoost:: For Colombia stock
Accuracy: 59.02%
f1 score is  0.5803278688524589
recall is  0.5901639344262295
precision score is  0.5966492523869573


/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
normalized_data = normalize_data(egypt_df)
X_train,y_train,X_test,y_test = split_train_test(normalized_data)
random_forest(X_train,y_train,X_test,y_test,"Egypt")
xg_boost(X_train,y_train,X_test,y_test,"Egypt")

Random Forest:: For Egypt stock
Accuracy: 0.48
XGBoost:: For Egypt stock
Accuracy: 54.84%
f1 score is  0.5483870967741935
recall is  0.5483870967741935
precision score is  0.5483870967741935
